In [2]:
import numpy as np
import tensorflow as tf
from PIL import Image

# المسارات
model_paths = {
    "VGG16": r"C:/Users/LAP-STORE/Desktop/Amit/computer vesion/VGG16_model.tflite",
    "Inception": r"C:/Users/LAP-STORE/Desktop/Amit/computer vesion/inception_model.tflite",
    "ResNet": r"C:/Users/LAP-STORE/Desktop/Amit/computer vesion/ResNet_model.tflite"
}

class_names = ['Eosinophil', 'Lymphocyte', 'Monocyte', 'Neutrophil']

# تحميل كل موديل
interpreters = {}
for name, path in model_paths.items():
    interpreter = tf.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()
    interpreters[name] = interpreter

# تجهيز الصورة
def preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    return np.expand_dims(image, axis=0)

# توقع بموديل واحد
def predict_with_model(interpreter, image):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # تحقق من شكل الإدخال
    input_shape = input_details[0]['shape']
    if image.shape[1:3] != tuple(input_shape[1:3]):
        from PIL import Image
        image_resized = tf.image.resize(image, input_shape[1:3]).numpy()
    else:
        image_resized = image

    interpreter.set_tensor(input_details[0]['index'], image_resized)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])[0]
    return output

# دالة التنبؤ الذكي
def smart_ensemble(image_path):
    image = preprocess_image(image_path)

    best_model = None
    best_confidence = -1
    best_class_index = -1
    best_prediction = None

    for name, interpreter in interpreters.items():
        prediction = predict_with_model(interpreter, image)
        confidence = np.max(prediction)
        class_index = np.argmax(prediction)

        print(f"{name} -> {class_names[class_index]} (Confidence: {confidence:.2f})")

        if confidence > best_confidence:
            best_confidence = confidence
            best_class_index = class_index
            best_model = name
            best_prediction = prediction

    return {
        "model": best_model,
        "class": class_names[best_class_index],
        "confidence": best_confidence,
        "raw_probs": best_prediction
    }

# مثال تشغيل
image_path = r"C:/Users/LAP-STORE/Desktop/Amit/computer vesion/blood/dataset2-master/dataset2-master/images/TEST_SIMPLE/EOSINOPHIL/_2_1226.jpeg"
result = smart_ensemble(image_path)

print("\n✅ Final Result:")
print(f"Best Model: {result['model']}")
print(f"Predicted Class: {result['class']} (Confidence: {result['confidence']:.2f})")


VGG16 -> Eosinophil (Confidence: 1.00)
Inception -> Eosinophil (Confidence: 1.00)
ResNet -> Eosinophil (Confidence: 0.40)

✅ Final Result:
Best Model: Inception
Predicted Class: Eosinophil (Confidence: 1.00)
